In [2]:
import pandas as pd

In [3]:
import sqlite3 as sql

In [4]:
from pathlib import Path

In [5]:
import mysql.connector

In [6]:
from mysql.connector import connect, Error

In [7]:
from sqlalchemy import create_engine

In [8]:
import os

In [9]:
path='/home/dpv/Desktop/Olist/conf/.env'
if (os.environ.get("DB_TYPE") == None):
    from dotenv import load_dotenv
    #from config.definitions import ROOT_DIR
    #load_dotenv(os.path.join(ROOT_DIR, 'config', 'conf', '.env'))
    load_dotenv(path)


In [10]:
#PASSWORD='cocoStella##22'
PASSWORD=os.environ.get("MYSQL_PWD")



In [11]:
try:
    with connect(
        host="localhost",
        user='root',
        password=PASSWORD,
    ) as connection:
        print(connection)
except Error as e:
    print(e)

In [12]:
# show_db_query = "SHOW DATABASES"
# with connection.cursor() as cursor:
#     cursor.execute(show_db_query)
#     for db in cursor:
#         print(db)


In [17]:
import mysql.connector

config = {
  'user': 'root',
  'password': PASSWORD,
  'host': '127.0.0.1',
  'database': 'olist',
  'raise_on_warnings': True
}

db_conn = mysql.connector.connect(**config)

#db_conn.close()

In [18]:
# query="SHOW TABLES;"
# #cursor = conn.cursor(buffered=True,dictionary=true)
# with cnx.cursor(buffered=True) as cursor:
#     cursor.execute(query)
#     #connection.commit()

In [19]:
# #import mysql.connector
# USER='dpv'
# PASSWORD='cocoStella##22'
# config = {
#   'user': 'root',
#   'password': PASSWORD,
#   #'Ehost': '##.##.#.##',
#   'database': 'olist',
#   #'port': '####',
#   'raise_on_warnings': True,
# }

# cnx = mysql.connector.connect(**config)

# #cnx.close()

In [20]:
mycursor = db_conn.cursor()
 
mycursor.execute("Show tables;")
 
myresult = mycursor.fetchall()
 
for x in myresult:
    print(x)

('customers',)
('olist_geolocation_dataset',)
('olist_order_items_dataset',)
('olist_order_payments_dataset',)
('olist_order_reviews_dataset',)
('olist_orders_dataset',)
('olist_products_dataset',)
('olist_sellers_dataset',)
('product_category_name_translation',)


In [23]:
mycursor.execute("SELECT COUNT(*) FROM customers;")
myresult = mycursor.fetchall()
for x in myresult:
    print(x)

(99441,)


In [31]:
customers=pd.read_sql(
  """
        select *
        from customers
        
  """, db_conn)
geo=pd.read_sql(
  """
        select *
        from geolocation
        
  """, db_conn)
order_items=pd.read_sql(
  """
        select *
        from order_items
        
  """, db_conn)

order_payments=pd.read_sql(
  """
        select *
        from order_payments
        
  """, db_conn)

order_reviews=pd.read_sql(
  """
        select *
        from order_reviews
        
  """, db_conn)


orders=pd.read_sql(
  """
        select *
        from orders
        
  """, db_conn)


products=pd.read_sql(
  """
        select *
        from olist_products_dataset
        
  """, db_conn)


sellers=pd.read_sql(
  """
        select *
        from olist_sellers_dataset
        
  """, db_conn)


translations=pd.read_sql(
  """
        select *
        from product_category_name_translation
        
  """, db_conn)


tables = [customers, geo, order_items, order_payments,order_reviews,
orders,products,sellers,translations]



/home/dpv/mambaforge/envs/fastClean/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/dpv/mambaforge/envs/fastClean/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/dpv/mambaforge/envs/fastClean/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/dpv/mambaforge/envs/fastClean/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pan

In [26]:
for table in tables:
    print(table.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000163 entries, 0 to 1000162
Data columns (total 5 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   geolocation_zip_code_prefix  1000163 non-null  object 
 1   geolocation_lat              1000163 non-null  float64
 2   geolocation_lng              1000163 non-null  float64
 3   geolocation_city             1000163 non-n

In [28]:
pd.read_sql(
  """
        Select DISTINCT order_status
        from olist_orders_dataset

  """, db_conn)

,order_status
0,delivered
1,invoiced
2,shipped
3,processing
4,unavailable
5,canceled
6,created
7,approved


In [32]:
SQL_Query = pd.read_sql(
  """
        Select 
          c.customer_unique_id, 
          o.order_purchase_timestamp,
          p.payment_value

        from customers as c
        join orders as o on o.customer_id = c.customer_id 
        join order_payments as p on o.order_id = p.order_id
        Where NOT o.order_status= 'canceled'       


  """, db_conn)

df = pd.DataFrame(SQL_Query)

/home/dpv/mambaforge/envs/fastClean/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103222 entries, 0 to 103221
Data columns (total 3 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   customer_unique_id        103222 non-null  object 
 1   order_purchase_timestamp  103222 non-null  object 
 2   payment_value             103222 non-null  float64
dtypes: float64(1), object(2)
memory usage: 2.4+ MB


In [34]:
df.head()

,customer_unique_id,order_purchase_timestamp,payment_value
0,bf635aadfff238fd7e9b0d25e800d019,2017-09-25 16:38:46,26.58
1,55020cb68ae358a51969695657a54e9c,2018-06-11 14:59:24,340.08
2,d02cbb85434c84acabb8e81bbf064d3e,2018-01-14 18:49:14,48.91
3,cb409daad1fb6069e979463ca3de23dc,2017-07-18 13:32:25,154.14
4,23daf66dd86071f57d332fa5562afbe9,2018-05-23 16:37:35,20.75


In [18]:
#full_path=Path('/var/lib/mysql/')

In [19]:
#full_path.exists()

True

In [20]:
#path = Path('/var/lib/mysql/olist.db')

In [46]:
#path.exists()

In [47]:
#!ls full_path

In [48]:
#!ls path

In [49]:
#db_conn = sql.connect(path)